In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import requests
import io


URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

response = requests.get(URL)

if response.status_code == 200:
    df = pd.read_csv(io.BytesIO(response.content))
else:
    print(f"Failed to download the data. Status code: {response.status_code}")

#### 1. Develop a *Line chart* using the functionality of pandas to show how automobile sales fluctuate from year to year.

In [ ]:
sales_per_year = df.groupby('Year')['Automobile_Sales'].sum()

plt.figure(figsize=(10, 6))
plt.plot(
    sales_per_year.index, sales_per_year.values, marker="o", linestyle="-", color="b"
)
plt.title("Ventas de Automóviles por Año")
plt.xlabel("Año")
plt.ylabel("Ventas de Automóviles")
plt.grid()
plt.xticks(
    sales_per_year.index
)  
plt.tight_layout()
plt.show()

#### 2. Plot different lines for categories of vehicle type and analyse the trend to answer the question "Is there a noticeable difference in sales trends between different vehicle types during recession periods?"

In [ ]:
recession_data = df[df["Recession"] == 1]

sales_per_year_vehicle = (
    recession_data.groupby(["Year", "Vehicle_Type"])["Automobile_Sales"].sum().unstack()
)

plt.figure(figsize=(12, 8))
sales_per_year_vehicle.plot(kind="line", marker="o")
plt.title("Ventas de Automóviles por Tipo de Vehículo durante Períodos de Recesión")
plt.xlabel("Año")
plt.ylabel("Ventas de Automóviles")
plt.grid()
plt.legend(title="Tipo de Vehículo")
plt.tight_layout()
plt.show()

#### 3: Use the functionality of Seaborn Library to create a visualization to compare the sales trend per vehicle type for a recession period with a non-recession period

In [ ]:
sales_per_year_recession = (
    df.groupby(["Year", "Vehicle_Type", "Recession"])["Automobile_Sales"]
    .sum()
    .reset_index()
)

plt.figure(figsize=(12, 8))
sns.barplot(
    data=sales_per_year_recession,
    x="Year",
    y="Automobile_Sales",
    hue="Vehicle_Type",
    palette="deep",  
    ci=None,
)

plt.title(
    "Comparación de Ventas de Automóviles por Tipo de Vehículo: Períodos de Recesión vs. No Recesión"
)
plt.xlabel("Año")
plt.ylabel("Ventas de Automóviles")
plt.grid(axis="y") 
plt.legend(title="Tipo de Vehículo")
plt.tight_layout()
plt.show()

####  4. Use sub plotting to compare the variations in GDP during recession and non-recession period by developing line plots for each period

In [ ]:
gdp_per_year = df.groupby(["Year", "Recession"])["GDP"].mean().reset_index()

fig, axes = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

recession_data = gdp_per_year[gdp_per_year["Recession"] == 1]
axes[0].plot(
    recession_data["Year"],
    recession_data["GDP"],
    marker="o",
    color="red",
    linestyle="-",
)
axes[0].set_title("PIB Promedio durante Períodos de Recesión")
axes[0].set_ylabel("PIB")
axes[0].grid()

non_recession_data = gdp_per_year[gdp_per_year["Recession"] == 0]
axes[1].plot(
    non_recession_data["Year"],
    non_recession_data["GDP"],
    marker="o",
    color="green",
    linestyle="-",
)
axes[1].set_title("PIB Promedio durante Períodos No Recesivos")
axes[1].set_ylabel("PIB")
axes[1].grid()

plt.xlabel("Año")
plt.xticks(
    gdp_per_year["Year"].unique()
) 
plt.tight_layout()
plt.show()

#### 5. Develop a Bubble plot for displaying the impact of seasonality on Automobile Sales.

In [ ]:
x = df["Seasonality_Weight"]
y = df["Automobile_Sales"]
size = df["Advertising_Expenditure"]

plt.figure(figsize=(12, 8))
plt.scatter(x, y, s=size, alpha=0.5, c="blue", edgecolors="w", linewidth=2)

plt.title("Impacto de la Estacionalidad en las Ventas de Automóviles")
plt.xlabel("Peso de Estacionalidad")
plt.ylabel("Ventas de Automóviles")
plt.grid()

plt.tight_layout()
plt.show()

#### 6. Use the functionality of Matplotlib to develop a scatter plot to identify the correlation between average vehicle price relate to the sales volume during recessions.

In [ ]:
recession_data = df[df["Recession"] == 1]

average_price = recession_data.groupby("Year")["Price"].mean().reset_index()
sales_volume = recession_data.groupby("Year")["Automobile_Sales"].sum().reset_index()

combined_data = pd.merge(average_price, sales_volume, on="Year")

plt.figure(figsize=(10, 6))
plt.scatter(
    combined_data["Price"], combined_data["Automobile_Sales"], color="purple", alpha=0.6
)

plt.title(
    "Correlación entre Precio Promedio de Vehículos y Volumen de Ventas durante Recesiones"
)
plt.xlabel("Precio Promedio de Vehículos")
plt.ylabel("Volumen de Ventas de Automóviles")
plt.grid()

plt.tight_layout()
plt.show()

#### 7. Create a pie chart to display the portion of advertising expenditure of XYZAutomotives during recession and non-recession periods.

In [ ]:
ad_expenditure_summary = (
    df.groupby("Recession")["Advertising_Expenditure"].sum().reset_index()
)

labels = ["Recesión", "No Recesión"]
sizes = ad_expenditure_summary["Advertising_Expenditure"]
colors = ["lightcoral", "lightskyblue"]

plt.figure(figsize=(8, 8))
plt.pie(
    sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=140, shadow=True
)

plt.title(
    "Gasto Publicitario de XYZ Automotives durante Períodos de Recesión y No Recesión"
)
plt.axis("equal")

#### 8. Develop a pie chart to display the total Advertisement expenditure for each vehicle type during recession period

In [ ]:
recession_data = df[df["Recession"] == 1]

ad_expenditure_by_vehicle = (
    recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
    .sum()
    .reset_index()
)

labels = ad_expenditure_by_vehicle["Vehicle_Type"]

sizes = ad_expenditure_by_vehicle["Advertising_Expenditure"]

colors = plt.cm.Paired(range(len(labels)))

plt.figure(figsize=(10, 8))

plt.pie(
    sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=140, shadow=True
)

plt.title("Gasto Total en Publicidad por Tipo de Vehículo durante Períodos de Recesión")

plt.axis("equal")

plt.tight_layout()

plt.show()

#### 9. Develop a line plot to analyse the effect of the unemployment rate on vehicle type and sales during the Recession Period.

In [ ]:
recession_data = df[df["Recession"] == 1]

grouped_data = (
    recession_data.groupby(["Year", "Vehicle_Type"])
    .agg(
        total_sales=("Automobile_Sales", "sum"),
        avg_unemployment_rate=("unemployment_rate", "mean"),
    )
    .reset_index()
)

plt.figure(figsize=(12, 6))

for vehicle_type in grouped_data["Vehicle_Type"].unique():
    vehicle_data = grouped_data[grouped_data["Vehicle_Type"] == vehicle_type]
    plt.plot(
        vehicle_data["Year"],
        vehicle_data["total_sales"],
        marker="o",
        label=f"Vehículo: {vehicle_type}",
    )

plt.title(
    "Efecto de la Tasa de Desempleo en Ventas de Vehículos durante Períodos de Recesión"
)
plt.xlabel("Año")
plt.ylabel("Total de Ventas de Automóviles")
plt.xticks(vehicle_data["Year"].unique())
plt.legend(title="Tipo de Vehículo")
plt.grid()

plt.tight_layout()
plt.show()

#### 10. Create a Dash application and give it a meaningful title

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

dropdown_options = [
    {"label": "Yearly Statistics", "value": "Yearly Statistics"},
    {"label": "Recession Period Statistics", "value": "Recession Period Statistics"},
]
# List of years
year_list = [i for i in range(1980, 2024)]

app.layout = html.Div(
    [
        html.H1(
            "Automobile Sales Statistics Dashboard.",
            style={"textAlign": "left", "color": "#503D36", "font-size": "24px"},
        ),
        html.Div(
            [ 
                html.Label("Select Statistics:"),
                dcc.Dropdown(
                    id="dropdown-statistics",
                    options=dropdown_options,
                    value="Select Statistics",
                    placeholder="Select a report type",
                ),
            ]
        ),
        html.Div(
            dcc.Dropdown(
                id="select-year",
                options=[{"label": i, "value": i} for i in year_list],
                value="Select Year",
            )
        ),
        html.Div(
            [ 
                html.Div(
                    id="output-container",
                    className="chart-grid",
                    style={"display": "flex"},
                ),
            ]
        ),
    ]
)


@app.callback(
    Output(component_id="select-year", component_property="disabled"),
    Input(component_id="dropdown-statistics", component_property="value"),
)
def update_input_container(selected_statistic):
    if selected_statistic == "Yearly Statistics":
        return False
    else:
        return True


@app.callback(
    Output(component_id="output-container", component_property="children"),
    [
        Input(component_id="select-year", component_property="value"),
        Input(component_id="dropdown-statistics", component_property="value"),
    ],
)
def update_output_container(selected_year, selected_statistic):
    if selected_statistic == "Recession Period Statistics":
        recession_data = df[df["Recession"] == 1]

        yearly_rec = (
            recession_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        )
        R_chart1 = dcc.Graph(
            figure=px.line(
                yearly_rec,
                x="Year",
                y="Automobile_Sales",
                title="Average Automobile Sales fluctuation over Recession Period",
            )
        )

        average_sales = (
            recession_data.groupby("Vehicle_Type")["Automobile_Sales"]
            .mean()
            .reset_index()
        )
        R_chart2 = dcc.Graph(
            figure=px.bar(
                average_sales,
                x="Vehicle_Type",
                y="Automobile_Sales",
                title="Average Number of Vehicles Sold by Vehicle Type",
            )
        )

        exp_rec = (
            recession_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .sum()
            .reset_index()
        )
        R_chart3 = dcc.Graph(
            figure=px.pie(
                exp_rec,
                names="Vehicle_Type",
                values="Advertising_Expenditure",
                title="Total Expenditure Share by Vehicle Type during Recessions",
            )
        )

        unemployment_rate = (
            recession_data.groupby("Vehicle_Type")["unemployment_rate"]
            .mean()
            .reset_index()
        )
        R_chart4 = dcc.Graph(
            figure=px.bar(
                unemployment_rate,
                x="Vehicle_Type",
                y="unemployment_rate",
                title="Unemployment Rate on Vehicle Type",
            )
        )

        return [
            html.Div(
                className="chart-item",
                children=[html.Div(children=R_chart1), html.Div(children=R_chart2)],
                style={"display": "flex"},
            ),
            html.Div(
                className="chart-item",
                children=[html.Div(children=R_chart3), html.Div(R_chart4)],
                style={"display": "flex"},
            ),
        ]

    elif selected_year and selected_statistic == "Yearly Statistics":
        yearly_data = df[df["Year"] == selected_year]
        yas = yearly_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(
                yas, x="Year", y="Automobile_Sales", title="Yearly Automobile Sales"
            )
        )

        month = yearly_data.groupby("Month")["Automobile_Sales"].mean().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(
                month, x="Month", y="Automobile_Sales", title="Monthly Automobile Sales"
            )
        )

        avr_vdata = yearly_data.groupby("Year")["Automobile_Sales"].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(
                avr_vdata,
                x="Year",
                y="Automobile_Sales",
                title="Average Vehicles Sold by Vehicle Type in the year {}".format(
                    selected_year
                ),
            )
        )

        exp_data = (
            yearly_data.groupby("Vehicle_Type")["Advertising_Expenditure"]
            .mean()
            .reset_index()
        )
        Y_chart4 = dcc.Graph(
            figure=px.pie(
                exp_data,
                names="Vehicle_Type",
                values="Advertising_Expenditure",
                title="Total Advertisement Expenditure",
            )
        )
        return [
            html.Div(
                className="chart-item",
                children=[html.Div(Y_chart1), html.Div(Y_chart2)],
                style={"display": "flex"},
            ),
            html.Div(
                className="chart-item",
                children=[html.Div(Y_chart3), html.Div(Y_chart4)],
                style={"display": "flex"},
            ),
        ]

    else:
        return None

if __name__ == "__main__":
    app.run_server(debug=True)

Exception in thread Thread-1044 (<lambda>):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dash/dash.py", line 1857, in <lambda>
    target=lambda: _watch.watch(
                   ^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dash/_watch.py", line 35, in watch
    walk()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dash/_watch.py", line 20, in walk
    info = os.st